Task name: Keyword association


Context: When artists upload artworks to Redbubble, they often include keywords to help users find their work 
    (referred to as work 'tags'). Artists can tag their work using any words they think are relevant. When users search
    for artwork, their search queries are matched against these tags. As artists have discretion over the use of these
    keywords, it is useful to be able to 
a) suggest additional keywords to the artist, based on words that have already been used; and
b) identify additional (relevant) artworks for each user search.

Data: The data for this task are in csv format and accessible from these links: 
* Artist tags: csv of tags assigned by artists for an artwork ( ~100,000 rows)
* User search keywords: tsv of keywords people searched for in a visit ( ~921,773 rows)

Objective: Using the dataset provided, identify relationships between keywords that could be used to suggest
    keywords or works,as described in the context section above.

Other information:
We are not expecting you to achieve a 'final' answer as part of this exercise. Rather we are interested in the way that you approach this task, and how you communicate your approach to us. 
Accordingly, you may use any analytical technique that you think is appropriate, but you should explain:
 - why you have chosen the technique(s) you have chosen
 - how the technique(s) you use work (explain to a non-technical audience)
 - any data cleaning/processing undertaken as part of your approach.
Please present your explanations and solution in a word document no longer than two pages.
You may use any programming language to undertake this analysis, but should include program files/scripts that replicat
e the analysis.

Introduction:

This task is more related to 'Tag Recommendation'. There is a number of published research work in this
topic. In this note, a proof of concept is presented. 

The senario of usage is that when an artist provides her tag-list, the recommender will suggest a number of candidate tags for each individual tag. The candidate tags are ranked according to a score computed based on a certain formula.  We assume that a tag-list is a list of tags seperated by comma.  

Implementation details:

The indexer:
An important module of the system is the indexer. The data (user_keywords and the tag_sets) are indexed to speed up the lookup of a keyword or a tag. For the tag_set index, the key is an individual tag and the value is a list of tag-lists that contains that tag. The index uses the position of the tag-list in the data as an Id. A python implementation of the indexer is given in the function (indexer).


Tag Recommender:
The tag recommender is the algorithm that suggests candidate tags to the artist. In its basic implementation the tag recommender consists mainly of two modules, tag co-occurance module and tags aggregation and ranking module. 

Tag co-occurance:
The tag co-occurance module finds the the relationship between every pair of artist tags. This relationship is quantified by computing the co-occurance between two tags. It is defined by the ratio of number of tag-lists that have both tags over the number of tag-lists that have any of the both tags. The function (compute_co_occurance) implements this definision.

Tag Aggregation and Ranking:
The tag aggregation module, takes a query tag-list and returns all the tags that have co_occurance coffecient more than zero.  To evaluate the importance of the tag, the tag is scored according to a certain formula. For this implementation, the score of the tag is computed based on the popularity (frequency) of the tag amoung user keywords.

Notes:
This implementation is for proof of concept and not yet production ready. The concept can be further improved in different aspects: 
a - A better formula for the tag score.
b - To get more information about the provided tag-list for example, the tag-list is a painting, a t-shirt, a cup ...
c - Reducing the time of computation, by storing the result of computation.
d - Better cleansing and filtering of the data

I have chosen the above techniques based on my experience in similar problems. The above implementation is modular with independent functionality. 

In [1]:
# Import some ML libraries stuff
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from collections import defaultdict
from sklearn import metrics
import pandas as pd
import numpy
# Suppress some warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas", lineno=570)

In [2]:
#Load the data, drop empty lines
tags_df = pd.read_csv(r'.\tag_sets.csv', header=None, names=['keywords']).dropna() 
keywords_df = pd.read_csv(r'.\user_keywords.tsv', sep='\t').dropna() 

In [3]:
#Tokenizer function that splits at , 
#We need to consider to split at other world boundries
def tokenizer(text):
    return text.split(',')   

In [4]:
#Indexer: implementation of index generator
def indexer( data ):
    assert isinstance(data, list), "Expected list input"
    index = defaultdict(set)
    for i,d in enumerate(data):
        for t in tokenizer(d):
            index[t].add(i)
    return index

In [5]:
#Create indeces for the tags and the keywords

#Create artist tags
tag_index = indexer(tags_df['keywords'].tolist())

#Create user keywords index
keyword_index = indexer( keywords_df['keywords'].tolist() )

In [6]:
#Create count words vectorizer for the artist tags
count_vect = CountVectorizer( tokenizer=lambda t:tokenizer(t), stop_words='english' )
tag_counts = count_vect.fit_transform( tags_df['keywords'].tolist() ) 

#The most frequent tags used by artists
term_count = zip(count_vect.get_feature_names(), numpy.asarray(tag_counts.sum(axis=0)).ravel())
term_count = sorted(term_count,key=lambda x:x[1], reverse=True)
print ('The 10 most frequent tags')
term_count[:10]

The 10 most frequent tags


[(u'funny', 12445),
 (u'cool', 8747),
 (u'geek', 8627),
 (u'cute', 6863),
 (u'music', 6807),
 (u'nerd', 6544),
 (u'anime', 5996),
 (u'retro', 5921),
 (u'love', 4575),
 (u'cartoon', 4519)]

In [7]:
#Compute the co-occurance between two tags based on the index 
#The cofficient is defined as:
# co_occurance_cof = no. of artist tag-list which have both tags (intersection)/ no. of artist tag-list which \
#have any of the two tags (union)

def compute_co_occurance(tag1, tag2, index):
    tag_list1 = index.get(tag1)
    tag_list2 = index.get(tag2)
    if not all([tag_list1,tag_list2]):
        return 0
    tag_list_inter = set.intersection(tag_list1, tag_list2)
    tag_list_union = set.union(tag_list1, tag_list2)
    return len(tag_list_inter)/float(len(tag_list_union))

In [8]:
#Aggregate the candidates, compute the co_occurance coffecient between enquiry artist tag and all the other tags in the
#index
#Return the tags of co_occurance value of more than 0
def aggregate(artist_tag, tag_index):
    candidates = {}
    for tag in tag_index.iterkeys():
        cof = compute_co_occurance(artist_tag, tag, tag_index)
        if tag != artist_tag and cof != 0.:
            candidates.update({tag:{'co_occ_cof':cof}})
    return candidates

In [9]:
#Ranking the candidate tags. The tag is given a score based on its popularity amoung the users
def score_candidates(candidates, keyword_index):
    no_keywords = float( len(keyword_index.keys()) )
    for c in candidates.iterkeys():
        keyword_count = float( len( keyword_index.get(c,[]) ))
        candidates[c].update( {'user_count':keyword_count*1000./no_keywords} )
    return candidates

In [10]:
#Function to find candidate tags for artist tag-list
def find_candidates(artist_tag_list, tag_index):
    assert isinstance(artist_tag_list, str), 'Expected string input'
    candidates = {}
    threshold = 0.1
    artist_tag_list_ = tokenizer(artist_tag_list)
    for tag in artist_tag_list:
        candidates.update( aggregate(tag, tag_index) )
    candidates = {k:v for k,v in candidates.iteritems() if v['co_occ_cof'] >= threshold}
    return candidates

In [12]:
example = "rocket,uniform,villain,grunt,team rocket,team rocket grunt,pokemon"

candidates_ = find_candidates(example, tag_index)
scored_candidates = score_candidates(candidates_, keyword_index)
candidates = sorted(scored_candidates.items(),key=lambda x:(x[1]['co_occ_cof'],x[1]['user_count']))

In [14]:
candidates

[('whitney', {'co_occ_cof': 0.1, 'user_count': 0.01427170762408893}),
 ('timberland', {'co_occ_cof': 0.1, 'user_count': 0.021407561436133395}),
 ('platoon', {'co_occ_cof': 0.1, 'user_count': 0.04424229363467568}),
 ('ciara', {'co_occ_cof': 0.1, 'user_count': 0.05851400125876461}),
 ('grand theft auto',
  {'co_occ_cof': 0.10050251256281408, 'user_count': 0.7050223566299931}),
 ('j hope',
  {'co_occ_cof': 0.10101010101010101, 'user_count': 0.01427170762408893}),
 ('blige', {'co_occ_cof': 0.1111111111111111, 'user_count': 0.0}),
 ('rap monster',
  {'co_occ_cof': 0.11475409836065574, 'user_count': 0.05851400125876461}),
 ('taehyung',
  {'co_occ_cof': 0.11498257839721254, 'user_count': 0.05280531820912904}),
 ('rockstar',
  {'co_occ_cof': 0.11764705882352941, 'user_count': 0.11845517327993811}),
 ('jin',
  {'co_occ_cof': 0.11858974358974358, 'user_count': 0.027116244485768968}),
 ('wiiu',
  {'co_occ_cof': 0.11940298507462686, 'user_count': 0.0042815122872266785}),
 ('bangtan boys',
  {'co_o

In [26]:
candidates

[('whitney', {'co_occ_cof': 0.1, 'user_count': 1.427170762408893e-05}),
 ('timberland', {'co_occ_cof': 0.1, 'user_count': 2.1407561436133393e-05}),
 ('platoon', {'co_occ_cof': 0.1, 'user_count': 4.4242293634675684e-05}),
 ('ciara', {'co_occ_cof': 0.1, 'user_count': 5.851400125876461e-05}),
 ('grand theft auto',
  {'co_occ_cof': 0.10050251256281408, 'user_count': 0.0007050223566299931}),
 ('j hope',
  {'co_occ_cof': 0.10101010101010101, 'user_count': 1.427170762408893e-05}),
 ('blige', {'co_occ_cof': 0.1111111111111111, 'user_count': 0.0}),
 ('rap monster',
  {'co_occ_cof': 0.11475409836065574, 'user_count': 5.851400125876461e-05}),
 ('taehyung',
  {'co_occ_cof': 0.11498257839721254, 'user_count': 5.280531820912904e-05}),
 ('rockstar',
  {'co_occ_cof': 0.11764705882352941, 'user_count': 0.00011845517327993812}),
 ('jin',
  {'co_occ_cof': 0.11858974358974358, 'user_count': 2.7116244485768967e-05}),
 ('wiiu',
  {'co_occ_cof': 0.11940298507462686, 'user_count': 4.281512287226679e-06}),
 ('